# Machine Learning Foundation

## Section 2, Part e: Regularization LAB


## Learning objectives

By the end of this lesson, you will be able to:

*   Implement data standardization
*   Implement variants of regularized regression
*   Combine data standardization with the train-test split procedure
*   Implement regularization to prevent overfitting in regression problems


In [1]:
import piplite
await piplite.install(['tqdm', 'seaborn', 'pandas', 'numpy'])

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

np.set_printoptions(precision=3, suppress=True)

Matplotlib is building the font cache; this may take a moment.


In the following cell we load the data and define some useful plotting functions.


In [ ]:
np.random.seed(72018)



def to_2d(array):
    return array.reshape(array.shape[0], -1)


    
def plot_exponential_data():
    data = np.exp(np.random.normal(size=1000))
    plt.hist(data)
    plt.show()
    return data
    
def plot_square_normal_data():
    data = np.square(np.random.normal(loc=5, size=1000))
    plt.hist(data)
    plt.show()
    return data

### Loading in Boston Data


In [3]:
import pyodide_http as ph
ph.patch_all()
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/boston_housing_clean.pickle"
data = pd.read_pickle(url)
boston_data = data['dataframe']
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
boston_description = data['description']
print(boston_description)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

## Data standardization


**Standardizing** data refers to transforming each variable so that it more closely follows a **standard** normal distribution, with mean 0 and standard deviation 1.

The [`StandardScaler`](http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01#sklearn.preprocessing.StandardScaler) object in SciKit Learn can do this.


**Generate X and y**:


In [4]:
y_col = "MEDV"
X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Import, fit, and transform using `StandardScaler`**


In [5]:
X.shape

(506, 13)

In [6]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)
X_ss[:5]

array([[-0.418,  0.285, -1.288, -0.273, -0.144,  0.414, -0.12 ,  0.14 ,
        -0.983, -0.667, -1.459,  0.441, -1.076],
       [-0.415, -0.488, -0.593, -0.273, -0.74 ,  0.194,  0.367,  0.557,
        -0.868, -0.987, -0.303,  0.441, -0.492],
       [-0.415, -0.488, -0.593, -0.273, -0.74 ,  1.283, -0.266,  0.557,
        -0.868, -0.987, -0.303,  0.396, -1.209],
       [-0.415, -0.488, -1.307, -0.273, -0.835,  1.016, -0.81 ,  1.078,
        -0.753, -1.106,  0.113,  0.416, -1.362],
       [-0.41 , -0.488, -1.307, -0.273, -0.835,  1.229, -0.511,  1.078,
        -0.753, -1.106,  0.113,  0.441, -1.027]])

In [7]:
# Recheck
pd.DataFrame(data=X_ss, columns=X.columns).head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.417713,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.415269,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.415272,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.414680,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.410409,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


### Exercise:

Confirm standard scaling


In [8]:
#Hint:

a = np.array([[1, 2, 3], 
              [4, 5, 6]]) 
print(a) # 2 rows, 3 columns

[[1 2 3]
 [4 5 6]]


In [9]:
a.mean(axis=0) # mean along the *columns*

array([2.5, 3.5, 4.5])

In [10]:
a.mean(axis=1) # mean along the *rows*

array([2., 5.])

In [11]:
### BEGIN SOLUTION
X2 = np.array(X)
man_transform = (X2-X2.mean(axis=0))/X2.std(axis=0)
np.allclose(man_transform, X_ss)
### END SOLUTION

True

### Coefficients with and without scaling


In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
lr = LinearRegression()

y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [14]:
lr.fit(X, y)
print(lr.coef_) # min = -18

[ -0.107   0.046   0.021   2.689 -17.796   3.805   0.001  -1.476   0.306
  -0.012  -0.953   0.009  -0.525]


In [15]:
np.sum(lr.coef_)

np.float64(-13.993579615882936)

#### Discussion (together):

The coefficients are on widely different scales. Is this "bad"?


In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
s = StandardScaler()
X_ss = s.fit_transform(X)

In [18]:
lr2 = LinearRegression()
lr2.fit(X_ss, y)
print(lr2.coef_) # coefficients now "on the same scale"

[-0.92   1.081  0.143  0.682 -2.06   2.671  0.021 -3.104  2.659 -2.076
 -2.062  0.857 -3.749]


In [19]:
np.sum(lr.coef_)

np.float64(-13.993579615882936)

### Exercise:

Based on these results, what is the most "impactful" feature (this is intended to be slightly ambiguous)? "In what direction" does it affect "y"?

**Hint:** Recall from last week that we can "zip up" the names of the features of a DataFrame `df` with a model `model` fitted on that DataFrame using:

```python
dict(zip(df.columns.values, model.coef_))
```


In [20]:
pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1, ascending=False, key=abs)

,0,1
12,LSTAT,-3.748680
7,DIS,-3.104448
5,RM,2.670641
8,RAD,2.658787
9,TAX,-2.075898
10,PTRATIO,-2.062156
4,NOX,-2.060092
1,ZN,1.080981
0,CRIM,-0.920411
11,B,0.856640


Looking just at the strength of the standardized coefficients LSTAT, DIS, RM and RAD are all the 'most impactful'. Sklearn does not have built in statistical signifigance of each of these variables which would aid in making this claim stronger/weaker


### Lasso with and without scaling


We discussed Lasso in lecture.

Let's review together:

1.  What is different about Lasso vs. regular Linear Regression?
2.  Is standardization more or less important with Lasso vs. Linear Regression? Why?


In [21]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

#### Create polynomial features


[`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [22]:
# Polynomial Features >>> Standard Scaler
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)

**Note:** We use `include_bias=False` since `Lasso` includes a bias by default.


In [23]:
X_pf_ss = s.fit_transform(X_pf)

### Lasso


[`Lasso` documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [24]:
las = Lasso()
las.fit(X_pf_ss, y)
np.sum(las.coef_) 

np.float64(-0.8967378020435977)

In [25]:
# Test: Standard Scaler >>> Polynomial Features
h = s.fit_transform(X)
a = pf.fit_transform(h)
las.fit(a, y)
np.sum(las.coef_)

np.float64(-2.321113934287175)

**Note:**  
(`Polynomial Features` --> `StandardScaler` --> `Lasso`) is better because Lasso regression works best when all features are on the same scale. If polynomial expansion happens first, scaling ensures that all polynomial terms contribute equally to the regularization process.

### Exercise

Compare

*   Sum of magnitudes of the coefficients
*   Number of coefficients that are zero

for Lasso with alpha 0.1 vs. 1.

Before doing the exercise, answer the following questions in one sentence each:

*   Which do you expect to have greater magnitude?
*   Which do you expect to have more zeros?


In [26]:
### BEGIN SOLUTION
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

sum of coefficients: 26.172415115426773
number of coefficients not equal to 0: 23


In [27]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())
### END SOLUTION

sum of coefficients: 8.472405227760156
number of coefficients not equal to 0: 7


- Smaller α (0.1) → Less regularization, more nonzero coefficients.
- Larger α (1) → Stronger regularization, fewer nonzero coefficients (more feature selection).

With more regularization (higher alpha) we will expect the penalty for higher weights to be greater and thus the coefficients to be pushed down. Thus a higher alpha means lower magnitude with more coefficients pushed down to 0.


### Exercise: $R^2$


Calculate the $R^2$ of each model without train/test split.

Recall that we import $R^2$ using:

```python
from sklearn.metrics import r2_score
```


In [28]:
from sklearn.metrics import r2_score
r2_score(y,las.predict(X_pf_ss))

0.6513089260815352

#### Discuss:

Will regularization ever increase model performance if we evaluate on the same dataset that we trained on?


## With train/test split


#### Discuss

Are there any issues with what we've done so far?

**Hint:** Think about the way we have done feature scaling.

Discuss in groups of two or three.


In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

In [31]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)      # alpha = 1
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_test, y_pred)

0.6780325981174933

In [32]:
X_train_s = s.fit_transform(X_train)
las01.fit(X_train_s, y_train)   # alpha = 0.1
X_test_s = s.transform(X_test)
y_pred = las01.predict(X_test_s)
r2_score(y_test, y_pred)

0.7999261342846065

### Exercise

#### Part 1:

Do the same thing with Lasso of:

*   `alpha` of 0.001
*   Increase `max_iter` to 100000 to ensure convergence.

Calculate the $R^2$ of the model.

Feel free to copy-paste code from above, but write a one sentence comment above each line of code explaining why you're doing what you're doing.

#### Part 2:

Do the same procedure as before, but with Linear Regression.

Calculate the $R^2$ of this model.

#### Part 3:

Compare the sums of the absolute values of the coefficients for both models, as well as the number of coefficients that are zero. Based on these measures, which model is a "simpler" description of the relationship between the features and the target?


In [40]:
(s.fit_transform(X_test_s) == s.fit_transform(X_test_s)).any()

np.True_

In [43]:
# fit and transform for scaling purpose
X_train_s = s.fit_transform(X_train)

# should use only ".transform" for same scaling with the above one
X_test_s = s.transform(X_test)     

# Part 1
# create Lasso model hyperparameters
lasr = Lasso(alpha=0.001, max_iter=100000)  # Lasso Regression

# fit the model
lasr.fit(X_train_s, y_train)

# prediction
y_pred_lasr = lasr.predict(X_test_s)
print("R^2 score (Lasso):", r2_score(y_test, y_pred_lasr))


# Part 2
# create Linear Regression model
lr = LinearRegression()

# fit the model
lr.fit(X_train_s, y_train)

# prediction
y_pred_lr = lr.predict(X_test_s)
print("R^2 score (Linear):",r2_score(y_test, y_pred_lr))

# Part 3
print("Sum of coefficient (Lasso):", abs(lasr.coef_).sum())
print("Sum of coefficient (Linear):", abs(lr.coef_).sum())
print("No. of coefficients equal to zero (Lasso):", (lasr.coef_ == 0).sum())
print("No. of coefficients equal to zero (Linear):", (lr.coef_ == 0).sum())

R^2 score (Lasso): 0.8847893236874534
R^2 score (Linear): 0.8689110469231067
Sum of coefficient (Lasso): 435.5723229043781
Sum of coefficient (Linear): 1183.8918138675313
No. of coefficients equal to zero (Lasso): 14
No. of coefficients equal to zero (Linear): 0


In [34]:
### BEGIN SOLUTION

# Part 1

# Decreasing regularization and ensuring convergence
las001 = Lasso(alpha = 0.001, max_iter=100000)

# Transforming training set to get standardized units
X_train_s = s.fit_transform(X_train)

# Fitting model to training set
las001.fit(X_train_s, y_train)

# Transforming test set using the parameters defined from training set
X_test_s = s.transform(X_test)

# Finding prediction on test set
y_pred = las001.predict(X_test_s)

# Calculating r2 score
print("r2 score for alpha = 0.001:", r2_score(y_test, y_pred))


# Part 2

# Using vanilla Linear Regression
lr = LinearRegression()

# Fitting model to training set
lr.fit(X_train_s, y_train)

# predicting on test set
y_pred_lr = lr.predict(X_test_s)

# Calculating r2 score
print("r2 score for Linear Regression:", r2_score(y_test,y_pred_lr))


# Part 3
print('Magnitude of Lasso coefficients:', abs(las001.coef_).sum())
print('Number of coeffients not equal to 0 for Lasso:', (las001.coef_!=0).sum())

print('Magnitude of Linear Regression coefficients:', abs(lr.coef_).sum())
print('Number of coeffients not equal to 0 for Linear Regression:', (lr.coef_!=0).sum())
### END SOLUTION

r2 score for alpha = 0.001: 0.8847893236874534
r2 score for Linear Regression: 0.8689110469231067
Magnitude of Lasso coefficients: 435.5723229043781
Number of coeffients not equal to 0 for Lasso: 90
Magnitude of Linear Regression coefficients: 1183.8918138675313
Number of coeffients not equal to 0 for Linear Regression: 104


## L1 vs. L2 Regularization


As mentioned in the deck: `Lasso` and `Ridge` regression have the same syntax in SciKit Learn.

Now we're going to compare the results from Ridge vs. Lasso regression:


[`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [44]:
from sklearn.linear_model import Ridge

### Exercise

Following the Ridge documentation from above:

1.  Define a Ridge object `r` with the same `alpha` as `las001`.
2.  Fit that object on `X` and `y` and print out the resulting coefficients.


In [45]:
# Decreasing regularization and ensuring convergence
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r2_score(y_test, y_pred_r)
### END SOLUTION

0.8702996266542663

In [46]:
las001 # same alpha as Ridge above

Lasso(alpha=0.001, max_iter=100000)

In [47]:
las001.coef_

array([  0.   ,   0.   , -17.008,   2.592,   0.   ,  13.431,  10.062,
       -19.542,   9.237,   0.   ,   6.159,  17.087,  11.486,   1.208,
         0.219,  10.827,   2.185,  -7.106,   4.312,  -1.526,  -2.092,
        -9.669,   0.   ,   0.   ,  -1.177,   3.865,   0.378,   0.194,
        -0.296,  -3.478,   0.291,   0.72 ,  -0.795,  -0.74 ,   2.407,
        -0.892,   2.835,  -0.984,   3.864,  -0.963,   6.914,   6.2  ,
         4.197,   0.892,   2.028,   2.608,  -3.975,   2.574,  -4.568,
         2.086,  -2.   ,  -7.412,   1.604,   1.676,  -1.248,  -0.   ,
        -0.   ,   2.576,  -0.851,   3.206,  -0.   ,  -2.429,   9.276,
        -7.056,   0.   ,  -5.955,  -5.326,   0.531,   3.344,  -7.575,
        -0.   ,  -6.762, -10.2  ,  -6.428,  -2.835,  -9.561,   0.288,
         0.549,  11.094,  -6.731,  -1.161,  -2.301,  -3.555,   9.12 ,
        -0.014,  -2.384,  -2.042,  -1.691,   0.798, -20.822,  27.09 ,
         0.   ,  -1.527,  -6.793,  -5.084,  15.577,   0.   ,  -0.   ,
         0.569,  -3.

In [48]:
print(np.sum(np.abs(r.coef_)))
print(np.sum(np.abs(las001.coef_)))

print(np.sum(r.coef_ != 0))
print(np.sum(las001.coef_ != 0))

792.8673754993279
435.5723229043781
104
90


**Conclusion:** Ridge does not make any coefficients 0. In addition, on this particular dataset, Lasso provides stronger overall regularization than Ridge for this value of `alpha` (not necessarily true in general).


In [49]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

0.9076091395031429
0.9103503442034472


**Conclusion**: Ignoring issues of overfitting, Ridge does slightly better than Lasso when `alpha` is set to 0.001 for each (not necessarily true in general).


# Example: Does it matter when you scale?


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [51]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.6982083583132745

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [53]:
s = StandardScaler()
lr_s = LinearRegression()
X_train_s = s.fit_transform(X_train)
lr_s.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lr_s.predict(X_test_s)
r2_score(y_test, y_pred)

0.6982083583132745

**Conclusion:** It doesn't matter whether you scale before or afterwards, in terms of the raw predictions, for Linear Regression. However, it matters for other algorithms. Plus, as we'll see later, we can make scaling part of a `Pipeline`.


***

### Machine Learning Foundation (C) 2020 IBM Corporation
